The notebook is for pipeline all data preparation of the football match data

In [30]:
import re
import pandas as pd
import os
import numpy as np
from csv import reader
import plotly.express as px
import missingno as msno
import pickle

In [ ]:
# load pickle and read content
d = pickle.load(open('./ELO/elo_dict.pkl', 'rb'))

Functions to get total goals so far for home team and away team in each game. The dataframe will call the apply() so that it will loop all records in the dataframe. And it will filter all records which round < current round and seperated with home and away for each team. The filter records then can calculate home total goals and away total goals

In [31]:
def getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, team, round):
    # determine home or away and get the score 
    # get home game of the team
    home_pd = data[(data["League"]==league) & (data["Home_Team"]==team) & (data["Season"]==season) & (data["Round"]<round)]
    df_home_score_sofar =  home_pd['Result'].str.extract(r'(\d)-\d')
    home_total_score = df_home_score_sofar[0].astype('Int64').sum()

    # get away game of the team
    away_pd = data[(data["League"]==league) & (data["Away_Team"]==team) & (data["Season"]==season) & (data["Round"]<round)]
    df_away_score_sofar =  away_pd['Result'].str.extract(r'\d-(\d)')
    away_total_score = df_away_score_sofar[0].astype('Int64').sum()

    # calculate total goals
    return (home_total_score + away_total_score)


def fillWithTotalGoalSoFar(record, data):
    # get home team and away team and round
    league = record['League']
    season = record['Season']
    round = record['Round']
    hteam = record['Home_Team']
    ateam = record['Away_Team']
    
    home_goal_so_far = getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, hteam, round)
    away_goal_so_far = getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, ateam, round)

    return [home_goal_so_far, away_goal_so_far]

Function to get elo home and away for each record in the dataframe by apply()

In [32]:
def fillWithELO(link):
    if link not in d:
        return [pd.NA, pd.NA]
    else:
        return [d[link]['Elo_home'], d[link]['Elo_away']]

Function to get recent performance with apply() similarly

In [33]:
def findRecentPreviousRounds(currentRound, limit):
    if currentRound<=limit:
        return None
    else:
        r = []
        for l in range(limit):
            r.append(currentRound - (limit-l))
        return r


def findLeagueSeasonTeamRecentPreviousRounds(data, league, season, team, round):
    rounds = findRecentPreviousRounds(round, 6)         # by definition is 6, can change for optimization
    if rounds is None:
        return None

    previous_matches_pd =  data[(data["League"]==league) & ((data["Home_Team"]==team) | (data["Away_Team"]==team)) & (data["Season"]==season) & (data["Round"].isin(rounds))]
    recent_perf = 0
    for index, row in previous_matches_pd.iterrows():
        hteam = row['Home_Team']
        ateam = row['Away_Team']
        if hteam==team:
            recent_perf = recent_perf + (row['Home_Score']-row['Away_Score'])
        else:
            recent_perf = recent_perf + (row['Away_Score']-row['Home_Score'])

    return recent_perf


def fillWithRecentPerformance(record, data):
    # get home team and away team and round
    league = record['League']
    season = record['Season']
    round = record['Round']
    hteam = record['Home_Team']
    ateam = record['Away_Team']
    
    home_team_goal_diff = findLeagueSeasonTeamRecentPreviousRounds(data, league, season, hteam, round)
    away_team_goal_diff = findLeagueSeasonTeamRecentPreviousRounds(data, league, season, ateam, round)

    return [home_team_goal_diff, away_team_goal_diff]

In [34]:
# load all directory as league name list
dir = "./Results"
leagues = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

# loop to open csv
result_with_goal_sofar_pd = pd.DataFrame()
for league in leagues:
    print("process league: " + league + "...")
    league_folder = os.path.join(dir, league)
    csv_file_for_league = [os.path.join(league_folder, name) for name in os.listdir(league_folder) if name.endswith('.csv')]
    
    for csv_filename in csv_file_for_league:
        current_league_season_pd = pd.read_csv(csv_filename, skiprows=[0], names=["Home_Team", "Away_Team", "Result", "Link", "Season", "Round", "League"])

        # Divide result into home_score and away_score
        df_score =  current_league_season_pd['Result'].str.extract(r'(\d)-(\d)')
        current_league_season_pd.insert(loc=3, column="Home_Score", value=df_score[0].astype('Int64'))     # use Int64 as it support NaN
        current_league_season_pd.insert(loc=4, column="Away_Score", value=df_score[1].astype('Int64')) 

        if len(current_league_season_pd)>0:
            # get home team and away team total goal so far
            home_away_total_goal_sofar = current_league_season_pd.apply(fillWithTotalGoalSoFar, data=current_league_season_pd, axis=1)
            goal_so_far_list = np.array(home_away_total_goal_sofar.values.tolist())         # convert to list
            home_away_total_goal_sofar_pd = pd.DataFrame(goal_so_far_list, columns=["HOME_GOAL_SO_FAR", "AWAY_GOAL_SO_FAR"])    # convert to dataframe
            current_league_season_pd.insert(loc=5, column="HOME_TOTAL_GOAL_SO_FAR", value=home_away_total_goal_sofar_pd["HOME_GOAL_SO_FAR"].astype('Int64')) 
            current_league_season_pd.insert(loc=6, column="AWAY_TOTAL_GOAL_SO_FAR", value=home_away_total_goal_sofar_pd["AWAY_GOAL_SO_FAR"].astype('Int64'))     

            # merge with ELO
            result_elo_pd = current_league_season_pd['Link'].apply(fillWithELO)   
            elo_list = np.array(result_elo_pd.values.tolist())
            elo_df = pd.DataFrame(elo_list, columns=["ELO_HOME", "ELO_AWAY"])
            current_league_season_pd.insert(loc=8, column="ELO_HOME", value=elo_df["ELO_HOME"].astype('Int64')) 
            current_league_season_pd.insert(loc=9, column="ELO_AWAY", value=elo_df["ELO_AWAY"].astype('Int64')) 

            # get recent performance
            home_away_recent_perf = current_league_season_pd.apply(fillWithRecentPerformance, data=current_league_season_pd, axis=1)
            perf_list = np.array(home_away_recent_perf.values.tolist())
            home_away_perf_pd = pd.DataFrame(perf_list, columns=["HOME_LAST_6_GOAL_DIFF", "AWAY_LAST_6_GOAL_DIFF"])
            current_league_season_pd.insert(loc=7, column="HOME_LAST_6_GOAL_DIFF", value=home_away_perf_pd["HOME_LAST_6_GOAL_DIFF"].astype('Int64')) 
            current_league_season_pd.insert(loc=8, column="AWAY_LAST_6_GOAL_DIFF", value=home_away_perf_pd["AWAY_LAST_6_GOAL_DIFF"].astype('Int64')) 

            result_with_goal_sofar_pd = pd.concat([result_with_goal_sofar_pd, current_league_season_pd])

# export to csv
result_with_goal_sofar_pd.to_csv('cleaned_dataset.csv', index=False)

process league: championship...
process league: eerste_divisie...
process league: primeira_liga...
process league: ligue_1...
process league: segunda_division...
process league: 2_liga...
process league: serie_a...
process league: bundesliga...
process league: primera_division...
process league: ligue_2...
process league: premier_league...
process league: eredivisie...
process league: segunda_liga...
process league: serie_b...


In [39]:
# delete no value column
result_with_goal_sofar_pd.drop('Result', inplace=True, axis=1)
result_with_goal_sofar_pd.drop('Link', inplace=True, axis=1)

In [41]:
# reorder dataframe column
result_with_goal_sofar_pd.insert(0, 'League', result_with_goal_sofar_pd.pop('League'))
result_with_goal_sofar_pd.insert(1, 'Season', result_with_goal_sofar_pd.pop('Season'))
result_with_goal_sofar_pd.insert(2, 'Round', result_with_goal_sofar_pd.pop('Round'))
result_with_goal_sofar_pd.insert(5, 'ELO_HOME', result_with_goal_sofar_pd.pop('ELO_HOME'))
result_with_goal_sofar_pd.insert(6, 'ELO_AWAY', result_with_goal_sofar_pd.pop('ELO_AWAY'))

In [48]:
result_with_goal_sofar_pd.head(30)

,League,Season,Round,Home_Team,Away_Team,ELO_HOME,ELO_AWAY,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LAST_6_GOAL_DIFF,AWAY_LAST_6_GOAL_DIFF
0,championship,2021,1,Watford,Middlesbrough,65.0,60.0,1,0,0,0,None,None
1,championship,2021,1,Birmingham City,Brentford,52.0,59.0,1,0,0,0,None,None
2,championship,2021,1,Wycombe Wanderers,Rotherham United,41.0,48.0,0,1,0,0,None,None
3,championship,2021,1,AFC Bournemouth,Blackburn Rovers,63.0,57.0,3,2,0,0,None,None
4,championship,2021,1,Barnsley,Luton Town,47.0,50.0,0,1,0,0,None,None
5,championship,2021,1,Bristol City,Coventry City,58.0,45.0,2,1,0,0,None,None
6,championship,2021,1,Cardiff City,Sheffield Wednesday,60.0,60.0,0,2,0,0,None,None
7,championship,2021,1,Derby County,Reading,61.0,54.0,0,2,0,0,None,None
8,championship,2021,1,Huddersfield Town,Norwich City,57.0,61.0,0,1,0,0,None,None
9,championship,2021,1,Millwall,Stoke City,55.0,65.0,0,0,0,0,None,None


In [54]:
result_with_goal_sofar_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146641 entries, 0 to 379
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   League                  146641 non-null  object
 1   Season                  146641 non-null  int64 
 2   Round                   146641 non-null  int64 
 3   Home_Team               146641 non-null  object
 4   Away_Team               146641 non-null  object
 5   ELO_HOME                122549 non-null  object
 6   ELO_AWAY                122549 non-null  object
 7   Home_Score              146545 non-null  Int64 
 8   Away_Score              146545 non-null  Int64 
 9   HOME_TOTAL_GOAL_SO_FAR  146641 non-null  int64 
 10  AWAY_TOTAL_GOAL_SO_FAR  146641 non-null  int64 
 11  HOME_LAST_6_GOAL_DIFF   121143 non-null  object
 12  AWAY_LAST_6_GOAL_DIFF   121141 non-null  object
dtypes: Int64(2), int64(4), object(7)
memory usage: 15.9+ MB
